To provide supervision for the prediction P in Eq. (3), we generate the pixel-level pseudo label from the initial CAM of the frozen backbone.

We first follow [Clip is also an efficient segmenter](https://arxiv.org/abs/2212.09506) to generate the initial CAM.


## Initial CAM Generation

For a given image $I$ with class label set $C_I$, the image is input to the frozen CLIP image encoder to generate the image feature map as $F \in \mathbb R^{d} \times (hw)$, after passing global average pooling, the feature vector $F_v \in \mathbb R \times 1$ is generated.

Meanwhile, the class labels set $C_I$, with the pre-defined background label set $C_{bg}$, are used to build text prompts using the text "a clear origami {*}", where * is the specific class label.

Then the text prompts are iput to the text encoder togenerate the feature map $F_t \in \mathbb R^{d \times (|C_I| + |C_{bg}|)}$.

Using $F_v$ and $F_t$, the distance is compute as:
$$
D = \frac{F_t F_v^T}{||F_t|| \cdot ||F_v||}
$$

Then, the distance is passed to the softmax function to generate the class scores:
$$
S^c = \text{Sotfmax}(D / \tau)
$$
where $S^c$ is the classifcation score for class $c$, and $c \in \{C_{bg}, C_I\}$, $\tau$ is the temperature parameter.

Using GradCAM, we can generate the feature weight map for a specific class $c$ in the $k$th channel:
$$
w_c^k = \frac{1}{hw} \sum_{i=1}^h \sum_{j=1}^w \sum_{c'} \frac{\partial S^c}{\partial D^{c'}} \frac{\partial D^{c'}}{\partial F^{k}_{i,j}}
$$
where $c \in \{C_{bg}, C_I\}$ and $c' \in \{C_{bg}, C_I\}$

Finally, the initial CAM for the specific foreground class $c$ is computed as:
$$
M_{init}^c(i,j) = \text{ReLU}\left(\sum_{k=1}^d w_c^k F_{i, j}^k\right)
$$